In [41]:
import h5py
import db
import series

from pprint import pprint


def add_series_to_sql(file_name):
    
    
    hdf5 = hdf5.H5Core()
    hdf5.set_files(file_name)
    info = hdf5.get_file_info()
    pprint(info)
    
    nameset = ['id', 'series_name', 'series_num', 'facility', 'fridge_run', 'data_type',
           'data_purpose', 'nb_events', 'nb_dumps', 'timestamp', 'comment']
    keyset = {'data_type': 'run_type', 'data_purpose': 'run_mode' }
    dataset = {}
    
    for name in nameset:
        print(name)
        if name in keyset: 
            n = keyset[name]
        else: n = name
            
        if n in info:  
            dataset[name] = info[n]
            print("insert")
            
        else: dataset[name] = "NULL"
            
            
    dataset['nb_events']=info['adc1']['nb_events']
    dataset['series_name'] = file_name.split('/')[-1][:-11]
    
    server = MySQLCore('pylelab_daq')
    server.connect_test()
    
    server.insert('test', dataset)
    print(dataset)
    
    server.disconnect()

In [22]:
def add_group_to_sql(group_name, group_path):
    
    group = series.SeriesGroup(group_name, group_path)
    
    group_info, series_info = group.get_group_info(include_series_info=True)
    
    server = db.MySQLCore('pylelab_daq')
    server.connect_test()
    
    server.insert('groups', group_info)
    
    for s in series_info:
        series_info[s].pop('ADC_config')
        series_info[s].pop('detector_config')
        series_info[s].pop('connection_table')
        
        server.insert('series', series_info[s])
    
    server.disconnect()

In [18]:
def get_metadata(file_name):
    
    import hdf5
    
    hdf5 = hdf5.H5Core()
    hdf5.set_files(file_name)
    info = hdf5.get_file_info()
    #pprint(info)
    
    nameset = ['id', 'series_name', 'series_num', 'facility', 'fridge_run', 'data_type',
           'data_purpose', 'nb_events', 'nb_dumps', 'timestamp', 'comment']
    keyset = {'data_type': 'run_type', 'data_purpose': 'run_mode' }
    dataset = {}
    
    for name in nameset:
        if name in keyset: 
            n = keyset[name]
        else: n = name
            
        if n in info:  
            dataset[name] = info[n]
            print("insert")
            
        else: dataset[name] = "NULL"
            
            
    dataset['nb_events']=info['adc1']['nb_events']
    sname = file_name.split('/')[-1][:-11]
    dataset['series_name'] = sname
    dataset['series_num'] = int(''.join((ch if ch in '0123456789' else '') for ch in sname))
    
    return dataset

In [42]:
#add_group_to_sql('testdaq_I2_D20211108_T225300', '/sdata/runs/run12/raw')

group = series.SeriesGroup('testdaq_I2_D20211108_T225300', '/sdata/runs/run12/raw')
group.get_group_info()
    
#group_info, series_info = group.get_group_info(include_series_info=True)

AttributeError: module 'pytesdaq.io.hdf5' has no attribute 'H5Reader'

In [2]:
add_to_sql('iv_I2_D20210622_T191952_F0001.hdf5')

{'adc1': {'adc_channel_indices': 2,
          'adc_conversion_factor': array([[-3.7919285e-03,  3.1250820e-04,  3.7905338e-14,  3.9733319e-18]],
      dtype=float32),
          'connection2': array(['detector:PD2', 'tes:C', 'controller:feb1_C'], dtype='<U17'),
          'data_mode': 'trig-ext',
          'dataset_list': ['event_1',
                           'event_10',
                           'event_100',
                           'event_101',
                           'event_102',
                           'event_103',
                           'event_104',
                           'event_105',
                           'event_106',
                           'event_107',
                           'event_108',
                           'event_109',
                           'event_11',
                           'event_110',
                           'event_111',
                           'event_112',
                           'event_113',
                           '

          'event_334': {'event_id': 334,
                        'event_index': 334,
                        'event_num': 100334,
                        'event_time': 1624414836},
          'event_335': {'event_id': 335,
                        'event_index': 335,
                        'event_num': 100335,
                        'event_time': 1624414836},
          'event_336': {'event_id': 336,
                        'event_index': 336,
                        'event_num': 100336,
                        'event_time': 1624414836},
          'event_337': {'event_id': 337,
                        'event_index': 337,
                        'event_num': 100337,
                        'event_time': 1624414836},
          'event_338': {'event_id': 338,
                        'event_index': 338,
                        'event_num': 100338,
                        'event_time': 1624414836},
          'event_339': {'event_id': 339,
                        'event_index': 339,
          

In [19]:
server = MySQLCore('pylelab_daq')
server.connect_manual(host="localhost", port=3306, user="root", password="Rzn>ed_24")


Database successfully connected!


True

In [36]:
name = 'trigger_I2_D20210324_T104549_F0003.hdf5'
name.split('/')[-1][:-11]

'trigger_I2_D20210324_T104549'

In [30]:
#### FINAL LOOP ####

table_name = 'test'
#for file in folder:
fname = "trigger_I2_D20210324_T104549_F0003.hdf5"
dataset = get_metadata(fname)
sname = fname.split('/')[-1][:-11]
timestamp = int(''.join((ch if ch in '0123456789' else '') for ch in sname)[1:])

print(timestamp)

last = int(server.get_last(table_name))

if timestamp >= last: 
    server_query = next((item for item in server.query(table_name, ["series_name"])
                         if item["series_name"] == sname), None)
    if server_query:
        server.modify(table_name, dataset)
        print("mod")
    else:
        server.insert(table_name, dataset)
        print("add")
        
        # for new files: iterate thru all files of a series, compare new number of dumps to old, edit num. dumps and num. events  


insert
insert
insert
20210324104549
mod


In [28]:
server.query("test", ["series_name"])

[{'series_name': 'trigger_I2_D20210324_T104549'},
 {'series_name': 'iv_I2_D20210622_T191952'},
 {'series_name': 'iv_I2_D20210622_T191952'},
 {'series_name': 'iv_I2_D20210622_T191952'},
 {'series_name': 'iv_I2_D20210622_T191952'},
 {'series_name': 'iv_I2_D20210622_T191952'},
 {'series_name': 'iv_I2_D20210622_T191952'},
 {'series_name': 'trigger_I2_D20210324_T104549'},
 {'series_name': 'trigger_I2_D20210324_T104549'},
 {'series_name': 'iv_I2_D20210622_T191952'},
 {'series_name': 'trigger_I2_D20210324_T104549'},
 {'series_name': 'trigger_I2_D20210324_T104549'}]

In [3]:
server.set_last('1234')

You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 'DECLARE @last INT' at line 1


[]